In [1]:
import sys
sys.path.append("../")
from args import all_args

usage: ipykernel_launcher.py [-h] [--model_decay_rate MODEL_DECAY_RATE]
                             [--lr LR] [--batch_size BATCH_SIZE]
                             [--data_dir DATA_DIR] [--device {cpu,cuda,auto}]
                             [--lmbd LMBD] [--n_epochs N_EPOCHS]
                             [--bad_limit BAD_LIMIT] [--n_series N_SERIES]
                             [--series_len SERIES_LEN] --output_dir OUTPUT_DIR
                             [--norm_type {none,minmax,standard}]
                             [--local_norm] [--valid_ratio VALID_RATIO]
                             [--test_ratio TEST_RATIO] [--fine_tuning]
                             {forecasting} ...
ipykernel_launcher.py: error: argument task_type: invalid choice: '/Users/chenghao.liu/Library/Jupyter/runtime/kernel-94026f94-3db3-455a-a590-778cd28cde64.json' (choose from 'forecasting')
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/chenghao.liu/opt/anaconda3/lib/python3.8/argparse.py", line 1800, in parse_known_args
    namespace, args = self._parse_known_args(args, namespace)
  File "/Users/chenghao.liu/opt/anaconda3/lib/python3.8/argparse.py", line 2009, in _parse_known_args
    stop_index = consume_positionals(start_index)
  File "/Users/chenghao.liu/opt/anaconda3/lib/python3.8/argparse.py", line 1965, in consume_positionals
    take_action(action, args)
  File "/Users/chenghao.liu/opt/anaconda3/lib/python3.8/argparse.py", line 1858, in take_action
    argument_values = self._get_values(action, argument_strings)
  File "/Users/chenghao.liu/opt/anaconda3/lib/python3.8/argparse.py", line 2399, in _get_values
    self._check_value(action, value[0])
  File "/Users/chenghao.liu/opt/anaconda3/lib/python3.8/argparse.py", line 2446, in _check_value
    raise ArgumentError(action, msg % args)
argparse.ArgumentError: argument task_type: invalid choice: '/Users/chenghao.l

TypeError: object of type 'NoneType' has no len()

In [2]:
import torch
a = torch.rand(3,2,4)
a

tensor([[[0.9249, 0.4316, 0.7377, 0.6504],
         [0.0025, 0.2595, 0.7984, 0.2474]],

        [[0.7503, 0.7272, 0.8989, 0.4891],
         [0.1854, 0.6921, 0.9823, 0.3477]],

        [[0.2275, 0.6934, 0.6390, 0.1764],
         [0.9089, 0.4785, 0.9676, 0.3096]]])

In [15]:
b = torch.tensor([1, 2, 3, 4])
c = b.repeat(3,1,1)
d = b.repeat(3,2,1)

In [16]:
a + b

tensor([[[1.9249, 2.4316, 3.7377, 4.6504],
         [1.0025, 2.2595, 3.7984, 4.2474]],

        [[1.7503, 2.7272, 3.8989, 4.4891],
         [1.1854, 2.6921, 3.9823, 4.3477]],

        [[1.2275, 2.6934, 3.6390, 4.1764],
         [1.9089, 2.4785, 3.9676, 4.3096]]])

In [17]:
a + d

tensor([[[1.9249, 2.4316, 3.7377, 4.6504],
         [1.0025, 2.2595, 3.7984, 4.2474]],

        [[1.7503, 2.7272, 3.8989, 4.4891],
         [1.1854, 2.6921, 3.9823, 4.3477]],

        [[1.2275, 2.6934, 3.6390, 4.1764],
         [1.9089, 2.4785, 3.9676, 4.3096]]])

In [18]:
a.shape

torch.Size([3, 2, 4])

In [20]:
c.shape

torch.Size([3, 1, 4])

In [21]:
d.shape

torch.Size([3, 2, 4])